https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

In [2]:
import spacy
import pandas as pd


In [8]:
dataFrame = pd.read_csv("ner_dataset.csv", encoding='latin-1')

In [29]:
df = dataFrame.head(56)

In [30]:
def process_data(df):
    df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")
    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    tag = df.groupby("Sentence #")["Tag"].apply(list).values
    return sentences, tag

In [31]:
process_data(df)

//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


(array([list(['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']),
        list(['Families', 'of', 'soldiers', 'killed', 'in', 'the', 'conflict', 'joined', 'the', 'protesters', 'who', 'carried', 'banners', 'with', 'such', 'slogans', 'as', '"', 'Bush', 'Number', 'One', 'Terrorist', '"', 'and', '"', 'Stop', 'the', 'Bombings', '.', '"']),
        list(['They', 'marched'])], dtype=object),
 array([list(['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']),
        list(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
        list(['O', 'O'])], dtype=object))

In [28]:
dataFrame.head(20)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
5,Sentence: 1,through,IN,O
6,Sentence: 1,London,NNP,B-geo
7,Sentence: 1,to,TO,O
8,Sentence: 1,protest,VB,O
9,Sentence: 1,the,DT,O


In [32]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [33]:
TRAIN_DATA = [
    ('Who is Nishanth?', {
        'entities': [(7, 15, 'PERSON')]
    }),
     ('Who is Kamal Khumar?', {
        'entities': [(7, 19, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    })
]

In [34]:
model = None
output_dir=Path("ner")
n_iter=100

In [35]:
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


In [36]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

  0%|          | 0/3 [00:00<?, ?it/s]

{'ner': 13.317753493785858}
{'ner': 12.119819581508636}
{'ner': 11.142244428396225}
{'ner': 10.148663520812988}
{'ner': 7.97254104167223}
{'ner': 7.1288202833384275}


  0%|          | 0/3 [00:00<?, ?it/s]

{'ner': 5.6652601305395365}
{'ner': 6.044148804736324}
{'ner': 6.832673414144665}
{'ner': 6.5430102195823565}
{'ner': 6.306748951901682}


  0%|          | 0/3 [00:00<?, ?it/s]

...

{'ner': 3.2222338219929534e-06}
{'ner': 0.00031975260760609307}
{'ner': 5.691784379430317e-08}
{'ner': 0.0003252306964599801}
{'ner': 1.5267682444378834e-05}
{'ner': 1.6868878884342997e-07}
{'ner': 2.405963902392367e-07}


  0%|          | 0/3 [00:00<?, ?it/s]

{'ner': 2.788262474326089e-07}
{'ner': 8.089289319441023e-07}
{'ner': 1.0511700901597741e-08}
{'ner': 1.8575960879516996e-10}
{'ner': 1.1304989067051499e-09}
{'ner': 9.736886482698609e-07}


100%|██████████| 3/3 [00:00<00:00, 80.68it/s]

{'ner': 1.1801765026897642e-08}
{'ner': 0.001168346081720464}
{'ner': 1.8647931893978814e-07}
{'ner': 0.00039354127274736643}
{'ner': 8.090516324911744e-06}
{'ner': 1.4073367038472735e-08}


In [37]:

for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('London', 'LOC'), ('Berlin', 'LOC')]
Entities [('Kamal Khumar', 'PERSON')]
Entities [('Nishanth', 'PERSON')]


In [38]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

Saved model to ner
